In [1]:
import pandas as pd
import warnings

# Ignoring FutureWarnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
df = pd.read_csv('csv/all_states_data.csv')
df.head()

,Código do Procedimento Ambulatorial,CID9 Principal (APAC ou BPA-I),Idade do paciente em anos,Sexo do paciente,Raça Cor do paciente,Valor Aprovado do procedimento,Indicador de Alta (APAC),Indicador de Óbito,Estado
0,604260016,N180,82,Mulher,Parda,50.40,Sim,Morto,Acre
1,604260016,N180,58,Homem,Parda,50.40,Sim,Morto,Acre
2,604260016,N180,70,Homem,Parda,10.08,Sim,Morto,Acre
3,604260016,N180,60,Mulher,Parda,50.40,Sim,Morto,Acre
4,604260016,N180,46,Mulher,Parda,50.40,Sim,Morto,Acre


In [3]:
# Grouping by 'Código do Procedimento Ambulatorial' and summing 'Valor Aprovado do procedimento'
codigo_valor = df.groupby('Código do Procedimento Ambulatorial')['Valor Aprovado do procedimento'].sum().reset_index()

# Convert 'Valor Aprovado do procedimento' to string with comma as decimal separator
codigo_valor['Valor Aprovado do procedimento'] = codigo_valor['Valor Aprovado do procedimento'].apply(lambda x: f"{x:.2f}".replace('.', ','))

codigo_valor.head()

,Código do Procedimento Ambulatorial,Valor Aprovado do procedimento
0,201010437,"184,76"
1,201010542,"97,00"
2,202010023,"66,33"
3,202010040,"3,63"
4,202010120,"344,10"


In [4]:
# Grouping by 'Código do Procedimento Ambulatorial' and summing 'Valor Aprovado do procedimento'
estado_valor = df.groupby('Estado')['Valor Aprovado do procedimento'].sum().reset_index()

# Convert 'Valor Aprovado do procedimento' to string with comma as decimal separator
estado_valor['Valor Aprovado do procedimento'] = estado_valor['Valor Aprovado do procedimento'].apply(lambda x: f"{x:.2f}".replace('.', ','))

estado_valor.head()

,Estado,Valor Aprovado do procedimento
0,Acre,"1406,16"
1,Alagoas,"105667,31"
2,Amapa,"2669,92"
3,Amazonas,"498534,36"
4,Bahia,"361532,25"


In [5]:
# Grouping by 'Código do Procedimento Ambulatorial' and summing 'Valor Aprovado do procedimento'
gender_valor = df.groupby('Sexo do paciente')['Valor Aprovado do procedimento'].sum().reset_index()

# Convert 'Valor Aprovado do procedimento' to string with comma as decimal separator
gender_valor['Valor Aprovado do procedimento'] = gender_valor['Valor Aprovado do procedimento'].apply(lambda x: f"{x:.2f}".replace('.', ','))

gender_valor.head()

,Sexo do paciente,Valor Aprovado do procedimento
0,Homem,"3260259,87"
1,Mulher,"2596596,58"


In [6]:
gender_valor.to_csv('csv/gender_valor.csv', index=False)
estado_valor.to_csv('csv/estado_valor.csv', index=False)
codigo_valor.to_csv('csv/codigo_valor.csv', index=False)

In [7]:
# Group by 'Idade do paciente em anos' and 'Indicador de Óbito', then count the occurrences
grouped = df.groupby(['Idade do paciente em anos', 'Indicador de Óbito']).size().unstack(fill_value=0)

# This will give you a DataFrame where the index is 'Idade do paciente em anos', 
# and the columns are 'Morto' and 'Vivo', with their respective counts.

# If you need to reset the index to have 'Idade do paciente em anos' as a column, you can do:
grouped = grouped.reset_index()

# Rename the columns for clarity if necessary
grouped.columns = ['Idade do paciente em anos', 'Frequência Mortos', 'Frequência Vivos']

grouped

,Idade do paciente em anos,Frequência Mortos,Frequência Vivos
0,0,87,0
1,1,74,0
2,2,107,0
3,3,110,0
4,4,88,0
...,...,...,...
109,115,6,0
110,116,2,0
111,122,180,0
112,123,184,0


In [8]:
# Define age bins and labels for the age groups
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']

# Filter for 'Indicador de Óbito' == 'Morto' with .copy() to avoid SettingWithCopyWarning
filtered_df = df[df['Indicador de Óbito'] == 'Morto'].copy()

# Use .loc to safely create 'Faixa Etária' column on the filtered DataFrame
filtered_df.loc[:, 'Faixa Etária'] = pd.cut(filtered_df['Idade do paciente em anos'], bins=bins, labels=labels, right=False)

# Group by 'Faixa Etária' and count occurrences, with observed=True to handle categorical data properly
faixa_etaria_counts = filtered_df.groupby('Faixa Etária', observed=True).size().reset_index(name='Frequency of Óbito')

faixa_etaria_counts

,Faixa Etária,Frequency of Óbito
0,0-10,892
1,11-20,2450
2,21-30,14683
3,31-40,31299
4,41-50,52894
5,51-60,72594
6,61-70,78342
7,71-80,51727
8,81-90,19901
9,91-100,2999


In [9]:
faixa_etaria_counts.to_csv('csv/faixa_idade.csv', index=False)

In [10]:
sum_valor = pd.read_csv('csv/sum_valor.csv')
average_idade = pd.read_csv('csv/average_idade.csv')
gender_valor = pd.read_csv('csv/gender_valor.csv')
raca_cor_freq = pd.read_csv('csv/raca_cor_freq.csv')

In [11]:
# Correcting the format of the 'Idade média em anos' column by replacing ',' with '.' and converting to float
average_idade['Idade média em anos'] = average_idade['Idade média em anos'].str.replace(',', '.').astype(float)

# Calculating the average age now with the corrected data
average_idade_corrected = average_idade['Idade média em anos'].mean()

In [12]:
# For sum_valor
sum_valor['Valor Aprovado do procedimento'] = sum_valor['Valor Aprovado do procedimento'].str.replace(',', '.').astype(float)

# For average_idade (assuming it might not need correction but checking if numeric conversion is required)
average_idade['Idade média em anos'] = pd.to_numeric(average_idade['Idade média em anos'], errors='coerce')

# Correcting gender_valor if needed
gender_valor['Valor Aprovado do procedimento'] = gender_valor['Valor Aprovado do procedimento'].str.replace(',', '.').astype(float)

# Now, we can proceed with the calculations again with the corrected data
total_sum_valor_corrected = sum_valor['Valor Aprovado do procedimento'].sum()

# Gender
proportion_homem_mulher_corrected = gender_valor.loc[gender_valor['Sexo do paciente'] == 'Homem', 'Valor Aprovado do procedimento'].sum() / \
                                    gender_valor.loc[gender_valor['Sexo do paciente'] == 'Mulher', 'Valor Aprovado do procedimento'].sum()

# For most affected etnia, we need to sum up all etnia frequencies across states and find the maximum
raca_cor_freq_melted = raca_cor_freq.melt(id_vars='Estado', value_vars=['Amarela', 'Branca', 'Indígena', 'Parda', 'Preta', 'Sem informação'], 
                                          var_name='Raca', value_name='Frequencia')
most_affected_raca = raca_cor_freq_melted.groupby('Raca')['Frequencia'].sum().idxmax()
most_affected_raca_freq = raca_cor_freq_melted.groupby('Raca')['Frequencia'].sum().max()

summary_table_corrected_with_comma_fix = {
    "Total sum_valor": total_sum_valor_corrected,
    "Média idade (average idade)": average_idade_corrected,
    "Proportion (Homem/Mulher)": proportion_homem_mulher_corrected,
    "Most affected Etnia": most_affected_raca,
    "Etnia Frequency": most_affected_raca_freq
}

summary_table_corrected_with_comma_fix

{'Total sum_valor': 5856856.450000001,
 'Média idade (average idade)': 55.23370370370371,
 'Proportion (Homem/Mulher)': 1.2555896803961746,
 'Most affected Etnia': 'Branca',
 'Etnia Frequency': 137168}

In [13]:
# Converting numerical values to strings with commas as decimal separators
summary_data_adjusted = {
    'Total sum_valor': ['{:.2f}'.format(5856856.450000001).replace('.', ',')],
    'Média idade (average idade)': ['{:.2f}'.format(55.23370370370371).replace('.', ',')],
    'Proportion (Homem/Mulher)': ['{:.2f}'.format(1.2555896803961746).replace('.', ',')],
    'Most affected Raça': ['Branca'],
    'Etnia Frequency': [137168]  # Keeping as integer, assuming no need for decimal formatting
}

# Converting to DataFrame
summary_df_adjusted = pd.DataFrame(summary_data_adjusted)

summary_df_adjusted

,Total sum_valor,Média idade (average idade),Proportion (Homem/Mulher),Most affected Raça,Etnia Frequency
0,"5856856,45","55,23","1,26",Branca,137168


In [14]:
summary_df_adjusted.to_csv('csv/summary_df.csv', index=False, decimal=',')

In [15]:
# Step 1: Find the most frequent Código do Procedimento Ambulatorial for each Estado
most_frequent_codes = df.groupby('Estado')['Código do Procedimento Ambulatorial'].agg(lambda x: x.mode()[0]).reset_index()

most_frequent_codes

,Estado,Código do Procedimento Ambulatorial
0,Acre,604470045
1,Alagoas,604470045
2,Amapa,604470045
3,Amazonas,604470045
4,Bahia,604470045
5,Ceara,604470045
6,Distrito Federal,604470045
7,Espirito Santo,604470045
8,Goias,604470045
9,Maranhao,604470045


In [16]:
# Step 2: Merge to get the Valor Aprovado for these codes
# This ensures we match the Estado and Código do Procedimento Ambulatorial to get the correct Valor Aprovado
result_df = pd.merge(most_frequent_codes, df.drop_duplicates(['Estado', 'Código do Procedimento Ambulatorial']), on=['Estado', 'Código do Procedimento Ambulatorial'], how='left')

In [17]:
# Step 3: Select the relevant columns to form the new DataFrame
proc_est_valor = result_df[['Estado', 'Código do Procedimento Ambulatorial', 'Valor Aprovado do procedimento']]

In [18]:
# Finding the top 5 unique 'Valor do procedimento aprovado' values
top_values = df['Código do Procedimento Ambulatorial'].unique()
top_values.sort()
top_5_unique_values = top_values[-5:]

# Filtering the DataFrame to only include rows with these top 5 values
df_filtered = df[df['Código do Procedimento Ambulatorial'].isin(top_5_unique_values)]

# Sorting by 'Valor do procedimento aprovado' to get the top values in order
df_sorted = df_filtered.sort_values(by='Valor Aprovado do procedimento', ascending=False)

# Dropping duplicates based on 'Valor do procedimento aprovado' to ensure uniqueness
df_unique_top = df_sorted.drop_duplicates(subset=['Valor Aprovado do procedimento'], keep='first')

# Renaming columns to 'Code' and 'Value'
final_df = df_unique_top[['Código do Procedimento Ambulatorial', 'Valor Aprovado do procedimento']]

pro_value = final_df.head()

# Replacing the specified code with "Médico Cirurgião"
pro_value['Código do Procedimento Ambulatorial'] = pro_value['Código do Procedimento Ambulatorial'].replace(803010087, 'Médico Cirurgião Geral')

pro_value

C:\Users\digui\AppData\Local\Temp\ipykernel_17000\280913733.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pro_value['Código do Procedimento Ambulatorial'] = pro_value['Código do Procedimento Ambulatorial'].replace(803010087, 'Médico Cirurgião Geral')


,Código do Procedimento Ambulatorial,Valor Aprovado do procedimento
8370,Médico Cirurgião Geral,3993.0
9991,Médico Cirurgião Geral,3448.5
9932,Médico Cirurgião Geral,3085.5
8452,Médico Cirurgião Geral,2904.0
10018,Médico Cirurgião Geral,2722.5


In [19]:
# Convert and replace
pro_value['Valor Aprovado do procedimento'] = pro_value['Valor Aprovado do procedimento'].apply(lambda x: f"{x:.1f}".replace('.', ','))
pro_value

C:\Users\digui\AppData\Local\Temp\ipykernel_17000\4276525710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pro_value['Valor Aprovado do procedimento'] = pro_value['Valor Aprovado do procedimento'].apply(lambda x: f"{x:.1f}".replace('.', ','))


,Código do Procedimento Ambulatorial,Valor Aprovado do procedimento
8370,Médico Cirurgião Geral,"3993,0"
9991,Médico Cirurgião Geral,"3448,5"
9932,Médico Cirurgião Geral,"3085,5"
8452,Médico Cirurgião Geral,"2904,0"
10018,Médico Cirurgião Geral,"2722,5"


In [20]:
pro_value.to_csv('csv/pro_value.csv', index=False, decimal=',')